In [1]:
# Dependencies
import numpy as np
import pandas as pd
import sqlite3

In [2]:
# Create/Connect to Database
db = sqlite3.connect('CapstoneDatabase.sqlite')

In [3]:
# Instantiate Cursor
c = db.cursor()

In [4]:
# Load csv into Pandas dataframe
lifestyle_data = pd.read_csv("CleanedData.csv", index_col = 0)
lifestyle_data.shape

(1000, 14)

In [5]:
# Convert Dataframe into a list of tuples
lifestyle_tuple = tuple(lifestyle_data.itertuples(index=False, name = None))

In [6]:
# Creating Lifestyle Table
c.execute("""CREATE TABLE lifestyle (
    patient_id text,
    age integer,
    gender integer,
    air_pollution integer,
    alcohol_use integer,
    dust_allergy integer,
    occupational_hazards integer,
    genetic_risk integer,
    chronic_lung_disease integer,
    balanced_diet integer,
    obesity integer,
    smoking integer,
    passive_smoking integer,
    cancer_severity integer
    


)




""")

In [7]:
c.executemany("INSERT INTO lifestyle VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)", lifestyle_tuple)

In [8]:
# Commit changes to the database
db.commit()

In [9]:
# Close database connection
db.close()